# Data Exploration

## Imports

In [1]:
import numpy as np
import pandas as pd
import requests

## USGS Wind Turbine Database
Example request response for wind turbine with 'case_id'=3038257. Response data contains turbine ID ('case_id'), rated capacity ('t_cap'), dimensions, coordinates ('xlong' and 'ylat'), and other relevant data. Calling the json() method on the response object gives a list of dictionaries, where each dictionary represents a turbine from the response.

In [9]:
base_path = 'https://eersc.usgs.gov/api/uswtdb/v1/'
resp = requests.get(f"{base_path}turbines?&case_id=eq.3038257")
resp.json()[0]

{'case_id': 3038257,
 'faa_ors': '19-022348',
 'faa_asn': '2011-WTE-22311-OE',
 'usgs_pr_id': 20977,
 't_state': 'IA',
 't_county': 'Story County',
 't_fips': '19169',
 'p_name': 'AG Land 4',
 'p_year': 2012,
 'p_tnum': 6,
 'p_cap': 9.6,
 't_manu': 'GE Wind',
 't_model': 'GE1.6-82.5',
 't_cap': 1600,
 't_hh': 80.0,
 't_rd': 82.5,
 't_rsa': 5345.62,
 't_ttlh': 121.3,
 't_conf_atr': 3,
 't_conf_loc': 3,
 't_img_date': '1/1/2013',
 't_img_srce': 'NAIP',
 'xlong': -93.3549,
 'ylat': 41.90419,
 'eia_id': None}